# **Importar bibliotecas**

In [1]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# **Fazer o request**  

In [2]:
url = "https://www.ibge.gov.br/indicadores.html"
pagina = requests.get(url)

# **Filtrar e selecionar o conteudo das tabelas**

In [3]:
soup = BeautifulSoup(pagina.content, "html.parser")

# Selecionar tabela
tabela = soup.find(class_="indicadores-tabela indicadores-economicos")

# Selecionar titulo
titulo = soup.find("th", class_="ultimo").text

# Selecionar index
rows = tabela.find_all(class_="nonsprite")
index = [id.text for id in rows]

# Selecionar conteudo
conteudo = tabela.find_all(class_=re.compile(r"table\-accordion.+"))

In [4]:
def filter_cols(col_class):
    """
    Funcao criada para selecionar as colunas da tabela
    """
    return ["vazio" if conteudo[n].find(class_=col_class) is None else conteudo[n].find(class_=col_class).text for n in list(range(0,20,2))]

In [5]:
# Coluna ultimo
ultimo = filter_cols("ultimo")

# Coluna anterior
anterior = filter_cols("desktop-tablet-only anterior")

# Coluna doze meses
doze_meses = filter_cols("desktop-tablet-only dozemeses")

# Coluna anual
anual = filter_cols("desktop-tablet-only ano")

# **Limpeza dos dados**

In [6]:
index = [re.sub(r"\n|^\r\n", "", id).strip() for id in index]

In [7]:
dataset = pd.DataFrame({"Último": ultimo, "Anterior": anterior, "Doze meses": doze_meses, "No ano": anual}, index = index).rename_axis(titulo, axis=0)

In [44]:
def cleaning_cols(col_index):
    """
    Funcao criada para limpar e padronizar os dados das colunas, 
    realizando selecao e substituicao de caracteres 
    e alterando seu dtype para float
    """
    coluna = dataset.iloc[:, col_index].str.findall(r"\d{0,4}\.?\d{1,5}\,\d\d?")
    coluna = coluna.apply(lambda n: n[0] if len(n) > 0 else '-')
    
    df = coluna.replace({'\.': '', ',': '.', '-': np.nan}, regex=True)
    
    df = df.astype("float")

    return df

In [47]:
# Coluna Ultimo
dataset["Último"] = cleaning_cols(0)

# Coluna Anterior
dataset["Anterior"] = cleaning_cols(1)

# Coluna Doze meses
dataset["Doze meses"] = cleaning_cols(2)

# Coluna No ano
dataset["No ano"] = cleaning_cols(3)

In [54]:
dataset

,Último,Anterior,Doze meses,No ano
Indicadores econômicos,,,,
IPCA (%),0.96,0.53,8.99,4.76
INPC (%),1.02,0.60,9.85,5.01
IPCA-15 (%),0.72,0.83,8.59,4.88
IPP (%),1.31,0.99,36.81,19.11
Custo do m² (%),1.89,2.46,22.60,13.49
Variação do PIB (%),1.00,1.10,3.80,1.00
PIB per capita (R$),33593.82,31843.95,NaN,NaN
Indústria (%),0.00,1.40,6.60,12.90
Comércio (%),1.70,2.70,5.90,6.70


# **Exportar arquivo em formato excel**

In [58]:
dataset.to_excel(f"planilha_ibge_{titulo}.xlsx")